# Archivos `.PARQUET` en Analítica y Big Data (Guía + Demo en Google Colab)

Este notebook sirve para **explicar qué es Parquet**, compararlo con **XLSX/CSV** y **leer un archivo `.parquet`** para generar un **reporte automático de variables** (tipos, nulos, ejemplos y estadísticos).

---

## Objetivos de aprendizaje
Al finalizar, podrás:

1. Explicar **qué es** un archivo Parquet y **por qué se usa** en Big Data.
2. Comparar Parquet vs **XLSX** vs **CSV** (ventajas, limitaciones, casos de uso).
3. Leer un `.parquet` en Colab con Python.
4. Generar un **diccionario/perfil de variables** del dataset.


## 1) ¿Qué es un archivo `.parquet`?

**Parquet** es un formato de almacenamiento de datos **columnar** (guarda los datos por columnas) y normalmente **comprimido**.  
Fue diseñado para **analítica** y **Big Data**: permite leer rápido, ahorrar espacio y mantener tipos de datos.

### ¿Por qué “columnar” importa?
- En formatos como CSV/Excel se almacenan datos principalmente **por filas**.
- En Parquet se guardan por **columnas**, lo cual permite:
  - Leer **solo** las columnas necesarias (mejor rendimiento).
  - Comprimir mejor (archivos más pequeños).
  - Procesar más rápido en motores analíticos (Spark, SQL engines, Python).

### ¿Para qué sirve Parquet?
- **Data Lakes** (S3/ADLS/GCS): almacenamiento analítico eficiente.
- Pipelines ETL/ELT: guardar “capas” de datos limpios/curados.
- Analítica y BI: acelerar agregaciones (KPI), joins y filtros.


## 2) Diferencias con otros formatos (XLSX, CSV, etc.)

### XLSX (Excel)
- ✅ Muy amigable: hojas, tablas, filtros, fórmulas.
- ✅ Ideal para trabajo manual, reportes y presentación.
- ❌ No está pensado para Big Data: se vuelve pesado/lento en volúmenes grandes.
- ❌ No es el mejor formato para automatizar pipelines.

### CSV
- ✅ Universal: se abre en casi cualquier herramienta.
- ✅ Útil para intercambio rápido.
- ❌ Menos eficiente: ocupa más espacio que Parquet y es más lento de leer en analítica.
- ❌ Tipos de datos frágiles: fechas/números pueden venir como texto.

### PARQUET
- ✅ Columnar + comprimido: más rápido y compacto.
- ✅ Mantiene mejor los tipos de datos.
- ✅ Excelente para analítica y Big Data (Spark/Python/SQL engines).
- ❌ No se abre “bonito” en Excel sin herramientas adicionales.

**Regla práctica:**
- Trabajo manual → **XLSX**
- Intercambio simple → **CSV**
- Analítica eficiente / Big Data → **PARQUET**


## 3) Preparación del entorno (instalar librerías)

En Colab, normalmente solo necesitas:
- `pandas` (tablas)
- `pyarrow` (lectura/escritura Parquet)

Ejecuta la celda siguiente.


In [ ]:
# Instalación (Colab)
!pip -q install pyarrow pandas


## 4) Cargar tu archivo Parquet en Colab

Sube el archivo `.parquet` desde tu computadora.  
Luego, el notebook detectará el archivo subido dentro de `/content/`.


In [ ]:
from google.colab import files
uploaded = files.upload()  # Selecciona tu archivo .parquet desde tu PC


## 5) Leer el archivo `.parquet` con pandas

La celda siguiente:
1. Busca el primer `.parquet` en `/content`
2. Lo lee con `pandas.read_parquet`
3. Muestra tamaño del dataset y primeras filas


In [ ]:
import pandas as pd
import os

parquets = [f for f in os.listdir("/content") if f.lower().endswith(".parquet")]
if not parquets:
    raise FileNotFoundError("No se encontró ningún archivo .parquet en /content. Sube el archivo primero.")

ruta = f"/content/{parquets[0]}"
print("Archivo detectado:", ruta)

df = pd.read_parquet(ruta)
print("Tamaño (filas, columnas):", df.shape)

df.head(20)


## 6) ¿Qué variables contiene? (Perfil/Diccionario automático)

Generaremos una tabla con:
- Nombre de variable
- Tipo de dato
- Nulos y % de nulos
- # de valores únicos
- Ejemplos de valores
- Estadísticos para variables numéricas (min, p25, mediana, promedio, p75, max)

Esto es muy útil como “**diccionario de datos**” inicial.


In [ ]:
import numpy as np

def perfil_variables(df: pd.DataFrame, max_ejemplos=5) -> pd.DataFrame:
    n = len(df)
    resumen = []

    for col in df.columns:
        s = df[col]
        nulos = int(s.isna().sum())
        pct_nulos = (nulos / n * 100) if n > 0 else 0.0
        unicos = int(s.nunique(dropna=True))

        # Ejemplos de valores no nulos
        ejemplos = s.dropna().astype(str).unique()[:max_ejemplos].tolist()
        ejemplos_txt = ", ".join(ejemplos) if ejemplos else ""

        fila = {
            "Variable": col,
            "Tipo": str(s.dtype),
            "Nulos": nulos,
            "% Nulos": round(pct_nulos, 2),
            "Únicos": unicos,
            "Ejemplos": ejemplos_txt
        }

        # Si es numérica, agregar estadísticos
        if pd.api.types.is_numeric_dtype(s):
            # Proteger si toda la columna es nula
            if nulos < n:
                fila.update({
                    "Min": float(np.nanmin(s)),
                    "P25": float(np.nanpercentile(s, 25)),
                    "Mediana": float(np.nanpercentile(s, 50)),
                    "Promedio": float(np.nanmean(s)),
                    "P75": float(np.nanpercentile(s, 75)),
                    "Max": float(np.nanmax(s))
                })
            else:
                fila.update({"Min": np.nan, "P25": np.nan, "Mediana": np.nan, "Promedio": np.nan, "P75": np.nan, "Max": np.nan})
        else:
            fila.update({"Min": "", "P25": "", "Mediana": "", "Promedio": "", "P75": "", "Max": ""})

        resumen.append(fila)

    return pd.DataFrame(resumen)

perfil = perfil_variables(df)
perfil


## 7) Resumen ejecutivo del dataset

Mostramos:
- Filas y columnas
- Top 10 variables con más nulos
- Conteo de tipos de datos


In [ ]:
print("=== RESUMEN EJECUTIVO ===")
print(f"Filas: {df.shape[0]:,}")
print(f"Columnas: {df.shape[1]:,}")

top_nulos = perfil.sort_values("% Nulos", ascending=False).head(10)[["Variable", "% Nulos", "Nulos"]]
print("\nTop 10 variables con más nulos:")
display(top_nulos)

tipos = perfil["Tipo"].value_counts()
print("\nTipos de datos (conteo de columnas):")
display(tipos)


## 8) (Opcional) Detección de columnas tipo fecha y conversión

En algunos datasets, las fechas llegan como texto.  
Esta celda intenta convertir columnas `object` a fecha si al menos el **80%** de valores calzan como fecha.


In [ ]:
def intentar_convertir_fechas(df: pd.DataFrame, umbral_exito=0.8):
    df2 = df.copy()
    convertidas = []
    for col in df2.columns:
        if df2[col].dtype == "object":
            parsed = pd.to_datetime(df2[col], errors="coerce", dayfirst=True)
            exito = parsed.notna().mean()  # proporción parseable
            if exito >= umbral_exito:
                df2[col] = parsed
                convertidas.append((col, round(exito*100, 1)))
    return df2, convertidas

df_conv, fechas = intentar_convertir_fechas(df)

if fechas:
    print("Columnas convertidas a fecha (columna, % éxito):")
    for c, p in fechas:
        print("-", c, ":", p, "%")
else:
    print("No se detectaron columnas de fecha para convertir automáticamente.")

df_conv.dtypes


## 9) (Opcional) Exportar el diccionario de datos a Excel/CSV

Esto es útil para entregar a estudiantes o documentar el proyecto.


In [ ]:
# Guardar el perfil/diccionario de variables
perfil.to_csv("diccionario_variables.csv", index=False)
perfil.to_excel("diccionario_variables.xlsx", index=False)

print("Archivos generados:")
print("- diccionario_variables.csv")
print("- diccionario_variables.xlsx")


## 10) Cierre: ¿Cuándo usar cada formato?

- **XLSX**: reportes, trabajo manual, presentaciones.
- **CSV**: intercambio simple y universal.
- **PARQUET**: analítica eficiente, Big Data, Data Lakes, procesamiento con Spark/SQL/Python.

### Flujo típico Big Data
1) Fuente (ERP/CRM/IoT) → extracción (CSV/JSON)
2) Limpieza/transformación → guardado en **Parquet** (Data Lake)
3) Analítica (Spark/Python/SQL) → resultados (tablas agregadas) → BI
